Вариант 17

In [10]:
import numpy as np

In [32]:
A = np.array([8, -5, 9, -8, 8, 8, -5, -8, 5, -4, -6, -2, 8, 3, 6, 6])
A = np.reshape(A, (4,4))
b = np.array([13, 38, 14, -95])
A


array([[ 8, -5,  9, -8],
       [ 8,  8, -5, -8],
       [ 5, -4, -6, -2],
       [ 8,  3,  6,  6]])

### Функции

LU разложение с выбором главного элемента:

In [33]:
def lu_pivot(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    swap_array = [] # в этот массив записываются перестановки строк матрицы
    maxi = 0 
    for j in range(N-1): 
        piv = u[0, :] #строка с максимальным по модулю элементом (pivot)
        for i in range(N): 
            if abs(u[i, j]) > abs(piv[j]):
                piv = u[i, :]
                maxi = i 
        if maxi > j:
            u[[j, maxi], :] = u[[maxi, j], :] #поднимаем строку с максимальным элементом наверх
            swap_array.append([j, maxi]) 
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j] 
        lam[j+1:, j] = -gamma 
        u = lam @ u #домножая на матрицу Лямбда зануляем соответствующий столбец 
        lam[j+1:, j] = gamma #это уже будет матрица обратная к Лямбда, из произведения этих матриц мы строим L
        L = L @ lam 
    return L, u, swap_array

Вычисление определителя: 

In [34]:
L, u, swap = lu_pivot(A)
print(A)
print(L @ u)
swap

[[ 8 -5  9 -8]
 [ 8  8 -5 -8]
 [ 5 -4 -6 -2]
 [ 8  3  6  6]]
[[ 8. -5.  9. -8.]
 [ 8.  8. -5. -8.]
 [ 5. -4. -6. -2.]
 [ 8.  3.  6.  6.]]


[]

In [35]:
def determinant(A):
    N = A.shape[0]
    L, u, s = lu_pivot(A)
    det_u = det_L = 1
    for i in range(N):
        det_u *= u[i, i]
        det_L *= L[i, i]
    det_A = det_u * det_L
    return det_A

Решение СЛАУ:

In [36]:
def solve_slau(A, b):
    N = A.shape[0]
    L, u, swap = lu_pivot(A)
    x = np.zeros(N)
    z = np.zeros(N)
    z[0] = b[0] # решаем систему Lz  = b, где z = Ux (прямой ход метода Гаусса)
    for i in range(1, N): 
        z[i] = b[i]
        for j in range(0, i):
            z[i] -= L[i, j] * z[j]
            
    x[N-1] = z[N-1] / u[N-1, N-1] #решаем систему Ux = z (обратный ход)
    for i in range(N-2, -1, -1): #идём с конца
        x[i] = z[i] / u[i, i]
        for j in range(i+1, N):
            x[i] -= (u[i, j] * x[j]) / u[i, i]
    return x
            

Нахождение обратной матрицы:

In [37]:
def invert(A):
    A1 = A.copy()
    N = A1.shape[0]
    E = np.eye(N)
    A_inv = np.zeros(A1.shape)
    for i in range(N):
        b = E[:, i]
        A_inv[:, i] = solve_slau(A1, b)
    return A_inv

### Тесты

Вычисление определителя:

In [7]:
L, u, swap = lu_pivot(A)
np.testing.assert_allclose(L @ u, A)
D = determinant(A)
print("Определитель матрицы:", D)

Определитель матрицы: 20050.0


Решение СЛАУ:

In [8]:
x = solve_slau(A, b)
np.testing.assert_allclose(A @ x, b)
print("x =", x)

x = [-4. -3. -1. -8.]


In [9]:
A_inv = invert(A)
E2 = A @ A_inv
eps = 1e-15
E2[np.abs(E2) < eps] = 0
print("Произведение матрицы на обратную к ней должно давать единичную: \n",E2)

Произведение матрицы на обратную к ней должно давать единичную: 
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
